In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Academic-Regulations.pdf")
text_documents = loader.load()
text_documents

[Document(metadata={'producer': 'Adobe PDF Library 23.6.156', 'creator': 'Acrobat PDFMaker 23 for Word', 'creationdate': '2023-12-19T17:49:44+05:30', 'author': 'Admin', 'company': '', 'moddate': '2024-02-13T12:16:38+05:30', 'sourcemodified': 'D:20231219121514', 'source': 'Academic-Regulations.pdf', 'total_pages': 43, 'page': 0, 'page_label': '1'}, page_content='F\nFCS Academic Regulations \nV\nersion 4.0 \nFFCS Academic Regulations - Version 4.0'),
 Document(metadata={'producer': 'Adobe PDF Library 23.6.156', 'creator': 'Acrobat PDFMaker 23 for Word', 'creationdate': '2023-12-19T17:49:44+05:30', 'author': 'Admin', 'company': '', 'moddate': '2024-02-13T12:16:38+05:30', 'sourcemodified': 'D:20231219121514', 'source': 'Academic-Regulations.pdf', 'total_pages': 43, 'page': 1, 'page_label': '2'}, page_content='CONTENTS PAGE NOS. \n1.0 Preamble 1 \n2.0 Scope 2 \n3.0 Admission 3 \n4.0 Academic System 3 \n4.1 Semester 3 \n4.2 Curriculum 4 \n4.3 Syllabus 4 \n4.4 Course Plan 5 \n4.5 Course Flowc

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_Splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_Splitter.split_documents(text_documents)
documents[:5]

[Document(metadata={'producer': 'Adobe PDF Library 23.6.156', 'creator': 'Acrobat PDFMaker 23 for Word', 'creationdate': '2023-12-19T17:49:44+05:30', 'author': 'Admin', 'company': '', 'moddate': '2024-02-13T12:16:38+05:30', 'sourcemodified': 'D:20231219121514', 'source': 'Academic-Regulations.pdf', 'total_pages': 43, 'page': 0, 'page_label': '1'}, page_content='F\nFCS Academic Regulations \nV\nersion 4.0 \nFFCS Academic Regulations - Version 4.0'),
 Document(metadata={'producer': 'Adobe PDF Library 23.6.156', 'creator': 'Acrobat PDFMaker 23 for Word', 'creationdate': '2023-12-19T17:49:44+05:30', 'author': 'Admin', 'company': '', 'moddate': '2024-02-13T12:16:38+05:30', 'sourcemodified': 'D:20231219121514', 'source': 'Academic-Regulations.pdf', 'total_pages': 43, 'page': 1, 'page_label': '2'}, page_content='CONTENTS PAGE NOS. \n1.0 Preamble 1 \n2.0 Scope 2 \n3.0 Admission 3 \n4.0 Academic System 3 \n4.1 Semester 3 \n4.2 Curriculum 4 \n4.3 Syllabus 4 \n4.4 Course Plan 5 \n4.5 Course Flowc

In [13]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents[:20], OllamaEmbeddings(model = "all-minilm"))


C:\Users\prana\AppData\Local\Temp\ipykernel_34328\1686328082.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  db = Chroma.from_documents(documents[:20], OllamaEmbeddings(model = "all-minilm"))


In [14]:
query = "All students seeking admission to various B.Tech and M.Tech programmes need to undertake national level computer based competitive examinations"
result = db.similarity_search(query)
result[0].page_content

'3 \n \n3. Admission \nAll students seeking admissi on to various B.Tech . and M.Tech . programmes need to undertake \nnational level computer based competitive examinations - VITEEE and VITMEE , respectively, \nconducted by the University once in a year, the dates of which are announced separately through \nmedia/university website. Selected students will be admitted into various programmes through \ncounseling. For all other programmes, students will be admitted based on their merit in addition to \nsatisfying specific admission criteria of the programme. The minimum qualifications essential for \nadmission to various programmes of the University will be stipulated and indicated in the Admissions \nbrochure released before the commencement of admission to various programmes every year.  \n \n4. Academic System \n \n4.1  Semester \nAll programmes under FFCS shall adopt a Semester system. There will be two semesters in an'

In [31]:
from langchain_community.llms import Ollama
llm = Ollama(model = "phi3")
llm

Ollama(model='phi3')

In [32]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are an academic assistant for technical programs. Answer questions regarding the university
using ONLY the following context. If the answer isn't in the context, say you don't know.

<context>
{context}
</context>

Question: {input}

Answer concisely and accurately in bullet points when appropriate:""")

In [33]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain =  create_stuff_documents_chain(llm, prompt)

In [34]:
retriever = db.as_retriever()

In [23]:
retrieved_docs = retriever.invoke("All students seeking admission to various B.Tech and M.Tech programmes need to undertake national level computer based competitive examinations")
print("Retrieved documents:", retrieved_docs)

Retrieved documents: [Document(metadata={'author': 'Admin', 'moddate': '2024-02-13T12:16:38+05:30', 'creationdate': '2023-12-19T17:49:44+05:30', 'company': '', 'producer': 'Adobe PDF Library 23.6.156', 'page_label': '6', 'creator': 'Acrobat PDFMaker 23 for Word', 'total_pages': 43, 'source': 'Academic-Regulations.pdf', 'sourcemodified': 'D:20231219121514', 'page': 5}, page_content='3 \n \n3. Admission \nAll students seeking admissi on to various B.Tech . and M.Tech . programmes need to undertake \nnational level computer based competitive examinations - VITEEE and VITMEE , respectively, \nconducted by the University once in a year, the dates of which are announced separately through \nmedia/university website. Selected students will be admitted into various programmes through \ncounseling. For all other programmes, students will be admitted based on their merit in addition to \nsatisfying specific admission criteria of the programme. The minimum qualifications essential for \nadmission

In [35]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [36]:
response = retrieval_chain.invoke({"input":"All students seeking admission to various B.Tech and M.Tech programmes need to undertake national level computer based competitive examinations"})

In [37]:
response['answer']

'- National Level Competitive Examinations required for admissions: VITEEE (B.Tech) & VITMEE (M.Tech). Conducted annually by the university, with dates announced on their website or media outlets. Selected candidates are admitted through counseling after clearing these exams and meeting specific admission criteria of each programme as per Admissions brochure released before programmes commence every year.\n- Courses in FFCS adopt a Semester system comprising two semesters with the option for additional credits from \'basket courses\' interdisciplinary options or backlog courses, benefiting both fast and slow learners to accommodate varying academic capabilities within each classification (Foundation Core - Discipline Core - Specialization Electives/Open Electives).\n- Students can register themselves with assistance through a Proctor. Bridge Courses for catching up are available in case of backlogs from previous semesters, allowing students the opportunity to retake and clear such cour

In [1]:
import sqlite3
print(sqlite3.sqlite_version)


3.45.3
